# DEDS Werkcollege 2.2

## Imports

In [21]:
import pyodbc
import sqlite3 as sql
import pandas as pd

## Connectie SSMS

In [26]:
DB = {
    "servername" : r"VISSIE\SQLEXPRESS",
    "database" : "SDM"}

export_conn = pyodbc.connect(f"""DRIVER={'SQL SERVER'};
                             SERVER={DB['servername']};
                             DATABASE={DB['database']};
                             Trusted_Connection=yes
                             """)
export_cursor = export_conn.cursor()

## Connecties SQL Databases (raw)

In [ ]:
connection_crm = sql.connect(r"..\..\..\data\raw\go_crm_train.sqlite")
connection_sales = sql.connect(r"..\..\..\data\raw\go_sales_train.sqlite")
connection_staff = sql.connect(r"..\..\..\data\raw\go_staff_train.sqlite")

## Dataframes
alle tabelnamen worden opgehaald door de create_dataframes_sql functie. Alle gelezen dataframes worden in een dictionary gestopt, met als key de tabel naam.

In [ ]:
def create_dataframes_sql(connection, db_type : str):
    dictionary : dict = {}
    query : str = ""
    key : str = ""
    
    if (db_type == "sqlite"):
        query = "SELECT name FROM sqlite_master WHERE type='table';"
        key = "name"
    elif (db_type == "ssms"):
        query = "SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE';"
        key = "TABLE_NAME"
    
    table_names = pd.read_sql(query, connection)
    
    for table in table_names[key].tolist():
        dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
    
    return dictionary

crm = create_dataframes_sql(connection_crm, "sqlite")
sales = create_dataframes_sql(connection_sales, "sqlite")
staff = create_dataframes_sql(connection_staff, "sqlite")
sdm = create_dataframes_sql(export_conn, "ssms")

C:\Users\jdvis_x5odeao\AppData\Local\Temp\ipykernel_24128\282968066.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_names = pd.read_sql(query, connection)
C:\Users\jdvis_x5odeao\AppData\Local\Temp\ipykernel_24128\282968066.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {table}", connection)
C:\Users\jdvis_x5odeao\AppData\Local\Temp\ipykernel_24128\282968066.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dictionary[table] = pd.read_sql(f"SELECT * FROM {ta

,RETURN_REASON_CODE,RETURN_DESCRIPTION_EN


## Query runners

In [89]:
from typing import Iterable


def query_remove(table_name : str):
    try:
        query = f"TRUNCATE {table_name}"
    except pyodbc.Error:
        print(query)
        return table_name
    
    return None

def create_add_query(row):
    query = ""
    columns = list(row.keys())
    
    for pos in range(len(columns)):
        if (pos == len(columns) - 1):
            query += f"{columns[pos]}) VALUES ("
        else:
            query += f"{columns[pos]}, "
    
    data = list(row)
    
    for pos in range(len(data)):
        if (pos == len(data) - 1):
            query += f"{data[pos]})"
        else:
            query += f"{data[pos]}, "
    
    return query

def query_add(table_name : str, table_data : pd.DataFrame):
    for index, row in table_data.iterrows():
        query : str = f"INSERT INTO {table_name} ("
        query += create_add_query(row)
        print(query)
        

## Leegmaken SDM Database

In [53]:
tables_sdm = list(sdm.keys())

while len(tables_sdm) > 0:
    for table in tables_sdm:
        temp = query_remove(table)
        
        tables_sdm.remove(table)
        
        if (temp != None):
            tables_sdm.add(temp)

print("All items are removed")

All items are removed


## Toevoegen data SDM Database

In [90]:
crm_keys = list(crm.keys())

for key in crm_keys:
    query_add(key, crm[key])

INSERT INTO age_group (AGE_GROUP_CODE, UPPER_AGE, LOWER_AGE) VALUES (1, 20, 0)
INSERT INTO age_group (AGE_GROUP_CODE, UPPER_AGE, LOWER_AGE) VALUES (2, 30, 21)
INSERT INTO age_group (AGE_GROUP_CODE, UPPER_AGE, LOWER_AGE) VALUES (3, 40, 31)
INSERT INTO age_group (AGE_GROUP_CODE, UPPER_AGE, LOWER_AGE) VALUES (4, 50, 41)
INSERT INTO age_group (AGE_GROUP_CODE, UPPER_AGE, LOWER_AGE) VALUES (5, 60, 51)
INSERT INTO age_group (AGE_GROUP_CODE, UPPER_AGE, LOWER_AGE) VALUES (6, 70, 61)
INSERT INTO country (COUNTRY_CODE, COUNTRY_EN, FLAG_IMAGE, SALES_TERRITORY_CODE) VALUES (1, France, F01, 6)
INSERT INTO country (COUNTRY_CODE, COUNTRY_EN, FLAG_IMAGE, SALES_TERRITORY_CODE) VALUES (2, Germany, F02, 6)
INSERT INTO country (COUNTRY_CODE, COUNTRY_EN, FLAG_IMAGE, SALES_TERRITORY_CODE) VALUES (3, United States, F03, 1)
INSERT INTO country (COUNTRY_CODE, COUNTRY_EN, FLAG_IMAGE, SALES_TERRITORY_CODE) VALUES (4, Canada, F04, 1)
INSERT INTO country (COUNTRY_CODE, COUNTRY_EN, FLAG_IMAGE, SALES_TERRITORY_CODE) 